In [1]:
# 들어가기 전에 아모레는 올리브영보다도 페이지 구조가 시시각각 바뀔수 있습니다.
# 전에는 잘됐던게 미스나고 안될수도 있어서 반드시 시도하기 전에 하나하나 확인해야합니다. 내가 지금 맞는 선택자를 쓰고있는지.

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_autoinstaller import install
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from math import ceil
import pandas as pd
import pyautogui
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10) # 10초 대기 설정

def close_ad_btn(driver):
    try:
        ad_btn = driver.find_element(By.CSS_SELECTOR, ".aiq-r_LBtE.qg-inweb-close")
        driver.execute_script("arguments[0].click();", ad_btn)
        time.sleep(2)  # 광고가 닫히는 것을 기다림
    except:
        print("광고 닫기 버튼을 찾지 못했습니다.")


# URL 설정
url = 'https://www.amoremall.com/kr/ko/display/category?searchCategory=%EB%A9%94%EC%9D%B4%ED%81%AC%EC%97%85&displayCategorySn=98' 


driver.get(url)

# 웹페이지를 열때 전체화면으로 열기
driver.maximize_window()

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3) # 스크롤이 완료될 때까지 대기

    # 새로운 높이를 확인
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새로운 높이가 이전 높이와 동일하다면 끝까지 스크롤된 것으로 간주
    if new_height == last_height:
        break

    last_height = new_height


# 광고 닫기 버튼 찾기
close_ad_btn(driver)

# 상품 목록 가져오기
products = driver.find_elements(By.CSS_SELECTOR, ".productCard.typeSmall a")
main_window_handle = driver.current_window_handle

reviews_data = []

print(len(products))
# 상품별로 반복
for i in range(len(products)):
    try:
        # 상품 목록 가져오기. 왜 위에 목록을 가져왔는데 또 가져오지? 할수도 있는데요.
        #  위에 가져온거는 반복을 돌리기 위해서 전체 제품을 전부 가져온거고
        # 안에서 다시 가져온 이유는 하나하나씩 for문 안에서 각각의 제품을 가져와서 크롤링해야 하기에 다시 가져온것입니다.
        products = driver.find_elements(By.CSS_SELECTOR, ".productCard.typeSmall a")
        product_link = products[i].get_attribute('href')

        # 새 탭에서 상품 링크 열기
        driver.execute_script(f"window.open('{product_link}', '_blank');")
        
        # 새로 열린 탭으로 전환
        driver.switch_to.window(driver.window_handles[1])



        # 브랜드명을 먼저 가져와서 '구독상품'이 포함되어 있는지 확인. (구독상품은 브랜드명부터 구독상품이기에.)
        product_preview = driver.find_element(By.CSS_SELECTOR, ".title .brand").text

        if '구독상품' in product_preview:
            print(f"'{product_preview}'으로 판단되어 건너뜁니다.")
            continue

        product_brand = driver.find_element(By.CSS_SELECTOR, ".title .brand").text
        product_name = driver.find_element(By.CSS_SELECTOR, ".title .name").text

        try:
            # 리뷰 탭이 로드될 때까지 최대 10초간 대기
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.tab[ap-click-name='상품상세_리뷰탭']")))
        except TimeoutException:
            print(f"{i+1}번째 상품의 상세 페이지가 10초 이내에 로드되지 않아 건너뜁니다.")
            driver.close()
            driver.switch_to.window(main_window_handle)
            continue
        
        review_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.tab[ap-click-name='상품상세_리뷰탭']")))
        driver.execute_script("arguments[0].click();", review_tab)
        time.sleep(2)

        review_count_text = driver.find_element(By.CSS_SELECTOR, "button.tab[ap-click-name='상품상세_리뷰탭'] span.inner").text
        review_count = int(review_count_text.split()[1].replace(',', ''))
        print(f"상품명: {product_name}의 리뷰 개수: {review_count}") # 리뷰 개수 출력


        # 저의 개인적인 생각에는 상품의 리뷰가 10개 이하라면 사실상 없다고 가정하는게 맞다고 생각해서 10개 이하로 한것입니다.
        # (사실 더보기버튼 누르기 전에 기본적으로 10개 리뷰가 있어서 10개 이하라면은 더보기버튼이 나타나지도 않기에 그런것입니다.)
        if review_count <= 10:
            print(f"상품명: {product_name}에 리뷰가 10개 이하입니다.(또는 없습니다). 다음 상품으로 넘어갑니다.")
                    # 새 탭 닫기
            driver.close()
            # 원래의 탭으로 돌아오기
            driver.switch_to.window(main_window_handle)
            continue # 다음 상품으로 이동

        # 광고 닫기 버튼 찾기. (아모레에서 리뷰탭 클릭후 뜨는 경우도 만들어놨습니다.)
        close_ad_btn(driver)

        max_reviews = min(1000, review_count)
        
        clicks_required = ceil(max_reviews / 10)
        
        while clicks_required > 0:
            # 상품 관련 이벤트가 있는지 확인. 아모레의 페이지 구조상 있다면 if 없다면 else 문에서 처리
            event_area = driver.find_elements(By.CSS_SELECTOR, ".article.eventArea")
            if event_area:
                more_btn_x, more_btn_y = 974, 555  # 가장 밑에까지 스크롤을 내렸을 경우에 관련 이벤트 부분 위에 리뷰 더보기가 위치하기에 관련 이벤트가 있을 경우에 좌표 세팅.
                # 반드시 본인 모니터에 따라서 변경이 필요합니다. (다 내 노트북과 똑같은 노트북을 쓰지는 않을 테니까.)
            else:
                more_btn_x, more_btn_y = 960, 915 # 관련 이벤트가 없다면 리뷰 더보기 버튼은 가장 밑에 존재하기에 좌표를 바꿈.
                # 역시 본인 모니터에 따라서 변경이 필요합니다.(사실 그냥 y만 변경해주면 됩니다. 마우스 커서를 봤을때 더보기버튼 위에 위치한다면 y값을 늘리시고 아래에 위치한다면 y값을 줄이셔야 됩니다.)

            try:
                driver.find_element(By.CSS_SELECTOR, "button.btnOutlined")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 일단 한번 끝가지 내려서 더보기가 화면에 보여 클릭할수 있게 만들기 위해서.
                time.sleep(2)
                pyautogui.click(x=more_btn_x, y=more_btn_y)  # 자바스크립트 클릭방식, click 함수를 이용해서 클릭했을때는 전부 안먹혔기에 마우스 움직임을 자동으로 제어해 클릭.
                time.sleep(2)
            except:
                break

            clicks_required -= 1

        # 리뷰 탭 내에서 맨 밑까지 스크롤. 이 while이 위에 스크롤하는게 있는데 왜 필요하냐할수도 있는데,
        # 위에 try 문에 있던 스크롤내리는거는 일단 처음 더보기버튼을 기준으로 끝까지 내린 후에 처음 더보기를 클릭할수 있게 만든거라서 더보기를 누르면 페이지의 끝이 바뀝니다. (더 내릴 스크롤이 생긴다는뜻.)
        # 그래서 끝나는거를 기억해서 계속 업데이트시켜야  하기에 이게 필요합니다.
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3) # 페이지 로딩 대기
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height


        # 모든 리뷰 카드 가져오기
        review_cards = driver.find_elements(By.CSS_SELECTOR, ".reviewCard")

        # 각 리뷰 카드를 순회하며 정보를 수집.
        for review_card in review_cards:
            username = review_card.find_element(By.CSS_SELECTOR, ".profileCard__userTitle").text
            if username == 'noname':
                print('탈퇴 회원입니다. 건너뜁니다.')
                print()
                continue
            star_wrap = review_card.find_element(By.CSS_SELECTOR, ".icoStarWrap").get_attribute('class')
            rating = int(star_wrap.split("star")[-1])
            review_date = review_card.find_element(By.CSS_SELECTOR, ".date").text
            review_text = review_card.find_element(By.CSS_SELECTOR, "p.txt").text

            # 정보 가져오기
            info_elements = review_card.find_elements(By.CSS_SELECTOR, ".profileCard__userDesc .info .item .txt")

            # 정보 분리. 정보가 없다면 결측치로 채우는 코드.
            age_info = info_elements[0].text if len(info_elements) > 0 else None
            gender_info = info_elements[1].text if len(info_elements) > 1 else None
            skin_type_info = info_elements[2].text if len(info_elements) > 2 else None
            skin_trouble_info = info_elements[3].text if len(info_elements) > 3 else None

            # 리뷰 정보를 딕셔너리로 저장
            review_data = {
                "브랜드명":product_brand,
                "상품명": product_name,
                "사용자 이름": username,
                "별점": rating,
                "리뷰작성날짜": review_date,
                "나이": age_info,
                "성별": gender_info,
                "피부타입": skin_type_info,
                "피부트러블": skin_trouble_info,
                "리뷰": review_text
            }

            reviews_data.append(review_data)

            print(f"브랜드명: {product_brand}")
            print(f"상품명: {product_name}")
            print(f"사용자 이름: {username}")
            print(f"별점: {rating}")
            print(f"리뷰작성날짜: {review_date}")
            print(f"나이: {age_info}")
            print(f"성별: {gender_info}")
            print(f'피부타입: {skin_type_info}')
            print(f'피부트러블: {skin_trouble_info}')
            print(f"리뷰: {review_text}\n")

        # 새 탭 닫기
        driver.close()
        
        # 원래의 탭으로 돌아오기
        driver.switch_to.window(main_window_handle)

    # 중간에 창이 닫힌 경우 수집을 중단하고 그동안 수집한 데이터를 저장.
    except Exception as e:
        print("에러 발생:", e)
        driver.close() # 현재의 창을 닫음
        driver.switch_to.window(main_window_handle) 
        continue

# 모든 상품을 순회한 후 데이터프레임 생성
reviews_df = pd.DataFrame(reviews_data)

# 중복된 행 제거
reviews_df = reviews_df.drop_duplicates(subset=['상품명', '리뷰'])

reviews_df['상품분류'] = '메이크업'
cols = ['상품분류'] + [col for col in reviews_df.columns if col not in ['상품분류']]
reviews_df = reviews_df[cols]

# '리뷰' 열에서 None 값을 NaN 값으로 변환
reviews_df['리뷰'].replace([None], pd.NA, inplace=True)

# '리뷰' 열의 NaN 값을 가진 행 제거
reviews_df.dropna(subset=['리뷰'], inplace=True)

reviews_df['리뷰'] = reviews_df['리뷰'].str.replace("\n", " ")
reviews_df['리뷰'] = reviews_df['리뷰'].str.replace("\t", " ")
reviews_df['리뷰'] = reviews_df['리뷰'].str.replace("\a", " ")
reviews_df['리뷰'] = reviews_df['리뷰'].str.replace("\b", " ")

reviews_df.reset_index(inplace=True, drop=True)

# 중간에 창이 닫히지 않고 100개의 제품을 전부 크롤링했을 경우 파일로 저장
reviews_df.to_csv('아모레크롤링_메이크업.csv', index=False) # 분류가 스킨케어가 아니라면 변경



# 드라이버 종료
driver.quit()